In [8]:
import random

import numpy as np
import pandas as pd
import sklearn
from sklearn import cluster                   # Algoritmos de clustering.
from sklearn import datasets                  # Crear datasets.
from sklearn import manifold                  # Algoritmos de reduccion de dimensionalidad.
from sklearn import decomposition             # Módulo de reducción de dimensionalidad.
from sklearn.utils import check_random_state  # Gestión de números aleatorios.

# Clustering jerárquico y dendrograma.
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.neighbors import NearestNeighbors

# UMAP para reducción de dimensionalidad.
import umap

# Visualizacion.
import matplotlib
import matplotlib.pyplot as plt
from sklearn.cluster import AffinityPropagation
import sys 
import os
sys.path.append(os.path.abspath('/home/fran/Escritorio/i3uex/education_drop_clustering/Code/analysis_and_modeling'))
import dunn_index
from sklearn import metrics

%matplotlib inline

In [9]:
analys_personal_data = pd.read_csv('../../../../Data/For_analysis_and_modeling/2nd_quadrimester/analys_personal_data.csv',sep='|')

In [10]:
analys_personal_data.head()

,expediente,cod_plan,des_plan,anio_apertura_expediente,abandona,convocatoria_acceso,des_acceso,nota_admision_def,sexo,edad_acceso,distance,becario,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median
0,2,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,6.836,D,25,300.6305,S,0.235294,0.176471,1.990064,4.175,5.7,3.84
1,3,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Formación Profesional,6.836,H,31,667.4115,S,0.312500,0.562500,1.566486,5.000,5.0,2.50
2,4,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Formación Profesional,6.836,D,28,91.0020,N,0.166667,0.444444,1.345324,3.825,5.2,2.53
3,5,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Selectividad,6.730,D,25,91.0020,N,0.263158,0.315789,1.850120,3.670,5.4,2.50
4,6,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,6.836,H,25,91.0020,N,1.000000,0.000000,1.339003,7.000,7.0,0.00


In [11]:
interval_cols = []

In [12]:
analys_personal_data_model = analys_personal_data.copy()

In [13]:
analys_personal_data_model.drop(['expediente','cod_plan','distance','becario','des_acceso'],axis=1,inplace=True)

In [14]:
note_bcket_array = np.array([5, 6.5, 8, 9.5, 10, 11.5, 13, 14])
analys_personal_data_model['nota_admision_def'] = pd.cut(
    analys_personal_data_model['nota_admision_def'], note_bcket_array, include_lowest=True)
interval_cols.append('nota_admision_def')

year_bcket_array = np.array([18, 20, 25, 30, 35, 40, 45, 50, 55, 60])
analys_personal_data_model['edad_acceso'] = pd.cut(
    analys_personal_data_model['edad_acceso'], year_bcket_array, include_lowest=True)
interval_cols.append('edad_acceso')


# distance_bcket_array = np.array([0, 0.1, 45, 90, 135, 250, 500, 1892])
# analys_personal_data_model['distance'] = pd.cut(
#     analys_personal_data_model['distance'], distance_bcket_array, include_lowest=True)
# interval_cols.append('distance')

note_bcket_array = np.array([0, 1.5, 3, 4.5, 6, 7.5, 9, 10])
analys_personal_data_model['cum_median'] = pd.cut(
    analys_personal_data_model['cum_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_median')

analys_personal_data_model['cum_pass_median'] = pd.cut(
    analys_personal_data_model['cum_pass_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_pass_median')

analys_personal_data_model['cum_fail_median'] = pd.cut(
    analys_personal_data_model['cum_fail_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_fail_median')

ratio_bcket_array = np.array([0, 0.25, 0.5, 0.75, 1])
analys_personal_data_model['cum_pass_ratio'] = pd.cut(
    analys_personal_data_model['cum_pass_ratio'], ratio_bcket_array, include_lowest=True)
interval_cols.append('cum_pass_ratio')

analys_personal_data_model['cum_absent_ratio'] = pd.cut(
    analys_personal_data_model['cum_absent_ratio'], ratio_bcket_array, include_lowest=True)
interval_cols.append('cum_absent_ratio')

std_bcket_array = np.array([0, 0.05,1.15,1.25,2.15,3.25])
analys_personal_data_model['std_deviation'] = pd.cut(
    analys_personal_data_model['std_deviation'], std_bcket_array, include_lowest=True)
interval_cols.append('std_deviation')


In [15]:
analys_personal_data_model.head()

,des_plan,anio_apertura_expediente,abandona,convocatoria_acceso,nota_admision_def,sexo,edad_acceso,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median
0,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,"(6.5, 8.0]",D,"(20.0, 25.0]","(-0.001, 0.25]","(-0.001, 0.25]","(1.25, 2.15]","(3.0, 4.5]","(4.5, 6.0]","(3.0, 4.5]"
1,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,"(6.5, 8.0]",H,"(30.0, 35.0]","(0.25, 0.5]","(0.5, 0.75]","(1.25, 2.15]","(4.5, 6.0]","(4.5, 6.0]","(1.5, 3.0]"
2,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,"(6.5, 8.0]",D,"(25.0, 30.0]","(-0.001, 0.25]","(0.25, 0.5]","(1.25, 2.15]","(3.0, 4.5]","(4.5, 6.0]","(1.5, 3.0]"
3,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,"(6.5, 8.0]",D,"(20.0, 25.0]","(0.25, 0.5]","(0.25, 0.5]","(1.25, 2.15]","(3.0, 4.5]","(4.5, 6.0]","(1.5, 3.0]"
4,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,"(6.5, 8.0]",H,"(20.0, 25.0]","(0.75, 1.0]","(-0.001, 0.25]","(1.25, 2.15]","(6.0, 7.5]","(6.0, 7.5]","(-0.001, 1.5]"


In [16]:
analys_personal_data_model.to_csv('./analys_personal_data_model_fsel.csv',index=False)

In [19]:
analys_personal_data_clust = pd.read_csv('./analys_personal_data_clust_fsel.csv',index_col=0)

In [20]:
analys_personal_data_clust.head()

,des_plan GRADO EN EDIFICACIÓN,des_plan GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIVILES,des_plan GRADO EN INGENIERÍA CIVIL - HIDROLOGÍA,des_plan GRADO EN INGENIERÍA CIVIL - TRANSPORTES Y SERVICIOS URBANOS,des_plan GRADO EN INGENIERÍA DE SONIDO E IMAGEN EN TELECOMUNICACIÓN,des_plan GRADO EN INGENIERÍA INFORMÁTICA EN INGENIERÍA DE COMPUTADORES,des_plan GRADO EN INGENIERÍA INFORMÁTICA EN INGENIERÍA DEL SOFTWARE,anio_apertura_expediente 2008-09,anio_apertura_expediente 2009-10,anio_apertura_expediente 2010-11,...,"cum_pass_median (4.5, 6.0]","cum_pass_median (6.0, 7.5]","cum_pass_median (7.5, 9.0]","cum_pass_median (9.0, 10.0]","cum_fail_median (-0.001, 1.5]","cum_fail_median (1.5, 3.0]","cum_fail_median (3.0, 4.5]","cum_fail_median (4.5, 6.0]",labels,abandona
1,False,True,False,False,False,False,False,False,True,False,...,True,False,False,False,False,False,True,False,2,0
2,False,True,False,False,False,False,False,False,True,False,...,True,False,False,False,False,True,False,False,2,1
3,False,True,False,False,False,False,False,False,True,False,...,True,False,False,False,False,True,False,False,2,0
4,False,True,False,False,False,False,False,False,True,False,...,True,False,False,False,False,True,False,False,2,1
5,False,True,False,False,False,False,False,False,True,False,...,False,True,False,False,True,False,False,False,2,0


In [21]:
def get_dunn_index(data, labels):
    import dunn_index
    from sklearn.metrics.pairwise import euclidean_distances
    distances = euclidean_distances(data)
    return dunn_index.dunn(labels,distances)

In [22]:
metrics.silhouette_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


0.08502392864524377

In [23]:
metrics.calinski_harabasz_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


171.93796556565388

In [24]:
metrics.davies_bouldin_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


2.9704274650330267

In [25]:
get_dunn_index(analys_personal_data_clust,analys_personal_data_clust['labels'])

0.34641016151377546

In [26]:
import pandas as pd
def get_impurity(dropout_cl0: pd.core.series.Series, dropout_cl1:pd.core.series.Series):
    impurity = -1
    n = len (dropout_cl0) + len (dropout_cl1)
    if len(dropout_cl0[dropout_cl0 == 0]) > len(dropout_cl0[dropout_cl0 == 1]):
        impurity = (len(dropout_cl0) - len(dropout_cl0[dropout_cl0 == 0]))/n
    else:
        impurity = (len(dropout_cl0) - len(dropout_cl0[dropout_cl0 == 1]))/n
    if len(dropout_cl1[dropout_cl1 == 0]) > len(dropout_cl1[dropout_cl1 == 1]):
        impurity = impurity + ((len(dropout_cl1) - len(dropout_cl1[dropout_cl1 == 0]))/n)
    else:
        impurity = impurity + ((len(dropout_cl1) - len(dropout_cl1[dropout_cl1 == 1]))/n)
        
    return impurity

In [27]:
get_impurity(analys_personal_data_clust[analys_personal_data_clust['labels']==1]['abandona'], analys_personal_data_clust[analys_personal_data_clust['labels']==2]['abandona'])

0.31979695431472077

In [19]:
# from apitep_utils.report import Report
# for label in analys_personal_data['labels'].cat.categories:
#     dset = analys_personal_data[analys_personal_data['labels'] == label]
#     report = Report()
#     report.generate_advanced(dset,'KMeans_1st_year_2_cl_'+str(label),sys.path[0]+ '/'+ '1st_year_cat_umap'+ str(label))